In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
ref_W, ref_H = 1024, 768

In [3]:
# INPUT PATHs
BG_PATH = 'E:/dataset/background/street_view/img/'
HM_PATH = 'E:/dataset/preprocessed/people/'
MASK_PATH = 'E:/dataset/preprocessed/mask/'

# OUTPUT PATHs
OUT_BG_PATH = 'E:/dataset/completed/background/'
OUT_MK_PATH = 'E:/dataset/completed/mask/'
OUT_MX_PATH = 'E:/dataset/completed/mixed/'
OUT_PP_PATH = 'E:/dataset/completed/people/'
OUT_MB_PATH = 'E:/dataset/completed/masked_background/'

In [4]:
bg_list, hm_list, mk_list = os.listdir(BG_PATH), os.listdir(HM_PATH), os.listdir(MASK_PATH)
bg_list[:5], hm_list[:5], mk_list[:5]

(['000001_3.jpg',
  '000003_4.jpg',
  '000011_5.jpg',
  '000014_5.jpg',
  '000017_3.jpg'],
 ['0_people.png',
  '10000_people.png',
  '10001_people.png',
  '10002_people.png',
  '10003_people.png'],
 ['0_mask.png',
  '10000_mask.png',
  '10001_mask.png',
  '10002_mask.png',
  '10003_mask.png'])

In [10]:
curr_img_count = int()
for i in tqdm(range(len(bg_list)), desc = 'Converting...'):
    background = cv2.cvtColor(cv2.imread(BG_PATH + bg_list[i]), cv2.COLOR_BGR2RGB)
    
    if background.shape[0] != ref_H or background.shape[1] != ref_W:
        background = cv2.resize(background, dsize = (ref_W, ref_H))
    
    mask_background = np.zeros_like(background)
    IH, IW = background.shape[0], background.shape[1]
    for j in range(1):
        people = cv2.cvtColor(cv2.imread(HM_PATH + hm_list[curr_img_count], cv2.IMREAD_UNCHANGED), cv2.COLOR_BGRA2RGBA)
        mask = cv2.imread(MASK_PATH + mk_list[curr_img_count])
        
        # print(f'alpha min: {people[:, :, 3].min()}, alpha max: {people[:, :, 3].max()}')
        
        # print(people.shape)
        
        PH, PW = people.shape[0], people.shape[1]
        prop = IH / PH
        
#         print(prop)
        
#         print(f'size: {(IH, int(PW * prop))}')
        
        people = cv2.resize(people, dsize = (int(PW * prop), IH))
        mask = cv2.resize(mask, dsize = (int(PW * prop), IH))
        
        random_flag, random_range, random_val = True if IW > people.shape[1] else False, int(), int()
        
#         print(f'{IW, people.shape[1]}\n{random_flag}')
        
        if random_flag:
            random_range = IW - people.shape[1]
            random_val = np.random.randint(0, random_range)
#             print(f'random val: {random_val}')
            people_resize = np.zeros([IH, IW, 4])
            mask_resize = np.zeros([IH, IW, 3])
#             print(f'test: {people_resize[:, random_val:random_val + people.shape[1], :].shape}')
            
            # print(f'before\nmask: {mask.shape}\npeople: {people.shape}\n---------')
            
            people_resize[:, random_val:random_val + people.shape[1], :] = people.copy()
            mask_resize[:, random_val:random_val + people.shape[1], :] = mask.copy()
            people = people_resize.copy().astype(int)
            mask = mask_resize.copy().astype(int)
#             print(f'after\nmask: {mask.shape}\npeople: {people.shape}\n---------')
            
#             plt.imshow(people)
#             plt.title(f'maxmin\n{people.max(), people.min()}')
#             plt.show()
        
        people = people[:, :min(IW, people.shape[1]), :]
        mask = mask[:, :min(IW, people.shape[1]), :]
        
#         plt.imshow(people)
#         plt.title(f'{people.shape}')
#         plt.show()
        
        replace_idx = np.where(people[:, :, 3] == 255)
        result = background.copy()
        
#         print(f'len: {len(replace_idx)}\n[0]len: {len(replace_idx[0])}\n[1]len: {len(replace_idx[1])}\ntype: {type(replace_idx)}\n[0]type: {type(replace_idx[0])}\n[1]type: {type(replace_idx[1])}')
        
        result[replace_idx] = people[:, :, :3][replace_idx]
                
        # plt.figure(figsize = (10, 50))
        # plt.subplot(1, 5, 1)
        # plt.imshow(background)
        # plt.title(f'background\n{background.shape}')
        # plt.axis('off')
        # plt.subplot(1, 5, 2)
        # plt.imshow(mask)
        # plt.title(f'mask\n{mask.shape}')
        # plt.axis('off')
        # plt.subplot(1, 5, 3)
        # plt.imshow(result)
        # plt.title(f'mixed\n{result.shape}')
        # plt.axis('off')
        # plt.subplot(1, 5, 4)
        # plt.imshow(people)
        # plt.title(f'people\n{people.shape}')
        # plt.axis('off')
        # plt.subplot(1, 5, 5)
        # plt.imshow(result)
        # plt.imshow(mask, alpha = 0.7)
        # plt.title('mask + mixed')
        # plt.axis('off')
        # plt.show()
        
        # print(people.shape)
        
        mask = np.float32(mask)
        people = np.float32(people)
        
#         print(f'test{people.min(), people.max()}')
        
        if people.shape[0] != ref_H or people.shape[1] != ref_W:
            continue
        
        cv2.imwrite(OUT_BG_PATH + f'{i}_{j}_background.png', cv2.cvtColor(background, cv2.COLOR_RGB2BGR))
        cv2.imwrite(OUT_MK_PATH + f'{i}_{j}_mask.png', cv2.cvtColor(mask, cv2.COLOR_RGB2BGR))
        cv2.imwrite(OUT_MX_PATH + f'{i}_{j}_mixed.png', cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
        cv2.imwrite(OUT_PP_PATH + f'{i}_{j}_people.png', cv2.cvtColor(people, cv2.COLOR_RGBA2BGRA))
        cv2.imwrite(OUT_MB_PATH + f'{i}_{j}_maskedbackground.png', cv2.cvtColor(background, cv2.COLOR_RGB2BGR) * (1 - mask))
        
        curr_img_count += 1

Converting...: 100%|██████████| 1253/1253 [04:21<00:00,  4.80it/s]
